In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import umap
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import pytorch_lightning as pl

from deeptime.models.oneclass.conv import ConvOCC
from deeptime.models.representation import ConvAutoEncoder
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
DATASET = 'ECGFiveDays'
ACTIVATION = 'sinl'
LABEL = 1
LATENT = 32

In [3]:
x_train, y_train = load_UCR_UEA_dataset(name=DATASET, split='train')
# Since the features from the sktime are instatiated as objects we have to manually convert them
y_train = np.array(y_train, dtype=np.int32)

sequence_length = x_train.values[0][0].shape[0]

x_test, y_test = load_UCR_UEA_dataset(name=DATASET, split='test')
y_test = np.array(y_test, dtype='int32') # Fixing the labels type

x_train_transformed = []
for val in x_train.values:
    x_train_transformed.append(val[0].tolist())
x_train = np.array(x_train_transformed)

x_test_transformed = []
for val in x_test.values:
    x_test_transformed.append(val[0].tolist())
x_test = np.array(x_test_transformed)

x_train = np.expand_dims(x_train_transformed, axis=1)
x_test = np.expand_dims(x_test_transformed, axis=1)

train_dataset = BaseDataset(x=x_train, y=y_train)
train_loader = DataLoader(train_dataset, batch_size=32)

model = ConvAutoEncoder(in_channels=1, in_features=sequence_length, latent_dim=LATENT)
model.load_state_dict(torch.load(f'../../../pretrain/representation/{DATASET}/conv_autoencoder-dim={LATENT}-l={LABEL}.pt'))

# x_train_ = x_train[y_train == label]
# y_train_ = y_train[y_train == label]

<All keys matched successfully>

In [4]:
y_test = np.array([1 if lbl == LABEL else -1 for lbl in y_test])
test_dataset = BaseDataset(x=x_test, y=y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
representations = []

for x, _ in train_loader:
    _, z = model(x)
    representations.extend(z.tolist())
representations = np.array(representations)

center = representations.mean(axis=0)
center

array([2.48609997e-03, 1.58402803e-01, 4.22164236e-02, 4.06682309e-03,
       7.19768268e-02, 1.19055475e-01, 0.00000000e+00, 1.11578008e-01,
       2.03621644e-01, 5.09013742e-01, 7.93226698e-02, 1.89969421e-03,
       3.77697364e-04, 1.86289558e-01, 6.32687860e-02, 5.83560253e-02,
       2.97538200e-01, 9.92069460e-02, 1.58992330e-02, 2.33221685e-01,
       2.88288505e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.07983170e-01, 0.00000000e+00, 2.23781298e-03, 0.00000000e+00,
       2.28914162e-02, 1.20229189e-01, 3.45366254e-03, 5.21974259e-02])

In [6]:
model = ConvOCC(
    in_channels=1,
    in_features=sequence_length,
    latent_dim=LATENT,
    radius=.2,
    learning_rate=5e-6
)

model.center = torch.from_numpy(center)

model.load_pretrained_weights(
    f'../../../pretrain/representation/{DATASET}/conv_autoencoder-dim={LATENT}-l={LABEL}.pt'
)

In [7]:
class LitProgressBar(pl.callbacks.TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(
            disable=True,
        )
        return bar

In [8]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator='gpu',
    devices=-1,
    callbacks=[LitProgressBar()]
)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | e    | Sequential | 4.8 M 
------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.030    Total estimated model params size (MB)


Epoch 66: 100%|████████| 28/28 [00:00<00:00, 82.80it/s, loss=6.31e-08, v_num=170, train_loss=0.000, val_f1_score=0.647]

In [9]:
trainer.test(model, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 84.02it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    test_accuracy_score    │    0.5656213704994193     │
│       test_f1_score       │    0.6472157446221736     │
│   test_precision_score    │     0.539599452492048     │
│     test_recall_score     │    0.8186782614215975     │
└───────────────────────────┴───────────────────────────┘

[{'test_f1_score': 0.6472157446221736,
  'test_accuracy_score': 0.5656213704994193,
  'test_recall_score': 0.8186782614215975,
  'test_precision_score': 0.539599452492048}]